In [1]:
import re
import getpass
# from encrypt import decryptString
import gmaps
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
from IPython.display import clear_output, display
# import Basin as basin
# import instantiationFimex as fimex
from parse import parse
# import metnoRequests as metno
# import instantiationFimex as fimex
# from shapely.wkb import loads
# from shapely.ops import cascaded_union
import yaml
import time
import os
import sys

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString
from prognos_tools.Basin import Basin


gmapsToken=b'gAAAAABctxJp5wE73qK6U5VieCi0WXNnNex4KxLZHutsESa8fW9v43lLa1Ag0qsxKFjXXo9MBAdvUpqJPW-QmCE0gH_Opf9g4xAG1VaI2WarO_xDZg44VLMCHkd_6O8ofgp8u4VuFBMr'
key = getpass.getpass('Password: ')
apiKey = decryptString(gmapsToken,key)
gmaps.configure(api_key=apiKey)
del key,apiKey

#Creating folder to store temporary files and downloaded data
saveFolder = './DownloadedData/'

args = [saveFolder,]
!!rm -rf {args[0]} && mkdir -p {args[0]}

Password:  ·······


[]

# Getting list of openDAP links with high-resolution (1km2) temperature and precipitation data

In [2]:
getList = False

baseURL = 'http://thredds.met.no/thredds/catalog/ngcd/version_19.03/catalog.html'
varsToGet = ['TX','TN','TG','RR']
model = 'type1'
listFolder = os.path.abspath('./' + model + 'Lists') #Folder where list of links will be saved

only_a_tags = SoupStrainer("a", href=True)

def getSoup(url,re_str):
    request=requests.get(url)
    soup=BeautifulSoup(request.content,'lxml',parse_only=only_a_tags)
    link_soup=soup.find_all('a',text=re.compile(re_str))
    links=[]
    for i in link_soup:
        links.append(urljoin(url,i['href']))
    return links

def getFileList(var):
    allLinks=[]
    for var_link in getSoup(baseURL,var):
        for model_link in getSoup(var_link,model):
            for year_link in getSoup(model_link,'^[0-9]{4}/$'):
                print('Processing {}'.format(year_link))
                for month_link in getSoup(year_link,'^[0-9]{2}/$'):
                    for day_link in getSoup(month_link,'\\.nc$'):
                        for opendap_link in getSoup(day_link,'^/thredds/dodsC/'):
                            allLinks.append(re.sub(r'\.html$', '', opendap_link))
    return allLinks
    

if getList:
    links = []
    !! rm -rf listFolder && mkdir listFolder
    for i in varsToGet:
        #Getting all links for variable
        links.append(getFileList(i))
        #Saving paths to file
        with open(os.path.join(listFolder , i + '_' + model + '.txt'), 'w') as f:
            for item in links[-1]:
                f.write("%s\n" % item)

## Querying extent of the basin
### Creating a VM with the geodatabase on it

In [3]:
machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'ljb-basin'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'geonorway'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/threddsBasin'
machineInfo['region'] = 'europe-west2-a'
machineInfo['machineType'] = 'n1-standard-4'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'

# #Folder to store results and intermediary scripts and basin info
# basinFolder = './basinResults/'

#Actually 
# basinGenerator = basin.Basin(instance,username,region,keyDir,machineType)
basinGenerator = Basin(machineInfo)
basinGenerator.instantiate(fabfile='fabfile_catchment.py')
print("The ip of the google cloud instance is {}".format(basinGenerator.ip))

#Testing connection to instance
basinGenerator.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**basinGenerator.machineInfo))

The ip of the google cloud instance is 35.246.18.90
Linux ljb-basin 4.9.0-6-amd64 #1 SMP Debian 4.9.88-1+deb9u1 (2018-05-07) x86_64

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
Pseudo-terminal will not be allocated because stdin is not a terminal.


### Getting basins extent

In [4]:
#Schema were processing will be done
schema = 'metno'

#Getting basin outlets from yaml file
stationsFile = '/home/jose-luis/Envs/get_metno_thredds/notebook/hans-fredrik.yaml'
stations = yaml.safe_load(open(stationsFile))
display(stations)

#Actually processing the basin
geoResultsFolder='./geoResults'
basinGenerator.getBasinLayers(stationsFile,schema,saveFolder=geoResultsFolder)
    
# Now we delete the basinGenerator vm
# basinGenerator.kill()

[{'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6558233,
   'longitude': 310091,
   'station_id': 3555,
   'station_name': 'Breidtjern'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6671439,
   'longitude': 332507,
   'station_id': 4203,
   'station_name': 'Bæreia'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6561016,
   'longitude': 183749,
   'station_id': 110,
   'station_name': 'Flåte'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6621713,
   'longitude': 282023,
   'station_id': 137,
   'station_name': 'Lyseren'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6622350,
   'longitude': 235553,
   'station_id': 5706,
   'station_name': 'Røysjø'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6620746,
   'longitude': 224023,
   'station_id': 5742,
   'station_name': 'Store Øyvannet'}},
 {'station': {'buffer': 30000,
   'epsg': 32633,
   'latitude': 6617213,
   'longitu

Gonna create the getBasin.sh file
./geoResults/getBasin.sh
fab -f fabfile_catchment.py processScript:./geoResults/getBasin.sh,./geoResults/box.txt
[35.246.18.90] Executing task 'processScript'
[35.246.18.90] put: ./geoResults/getBasin.sh -> getBasin.sh
[35.246.18.90] run: chmod +x getBasin.sh
[35.246.18.90] run: ./getBasin.sh
[35.246.18.90] out: NOTICE:  table "stations" does not exist, skipping
[35.246.18.90] out:  initializestations 
[35.246.18.90] out: --------------------
[35.246.18.90] out:  
[35.246.18.90] out: (1 row)
[35.246.18.90] out: 
[35.246.18.90] out:  addstations 
[35.246.18.90] out: -------------
[35.246.18.90] out:  
[35.246.18.90] out: (1 row)
[35.246.18.90] out: 
[35.246.18.90] out: NOTICE:  drop cascades to 7 other objects
[35.246.18.90] out: DETAIL:  drop cascades to table metno.stations
[35.246.18.90] out: drop cascades to table metno.demrast
[35.246.18.90] out: drop cascades to table metno.demshp
[35.246.18.90] out: drop cascades to view metno.flow
[35.246.18.90]

### Displaying basins

In [5]:
with open(os.path.join(geoResultsFolder,'gmaps.txt'), 'r') as f:
    lines = f.read().splitlines()
    
fig = gmaps.figure()
fig.add_layer(gmaps.geojson_layer(yaml.safe_load(lines[0])))

lines[2:] = [i.split(';') for i in lines[2:]]
outlets = [{"name": i[0], "area": i[3]} for i in lines[2:]]
locations = [(float(i[2]),float(i[1])) for i in lines[2:]]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Getting data from metno thredds servers
## Creating a virtual machine with the fimex utility

In [ ]:
#And now we can go about creating the vm with fimex on it
# instanceName = 'fimexin'
# username = 'jose-luis'
# keyDir = 'fimex'
# region = 'europe-north1-c'
# machineType = 'n1-standard-32'

machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'fimexin'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'fimex'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/fimex'
machineInfo['region'] = 'europe-north1-c'
machineInfo['machineType'] = 'n1-standard-32'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'

fimexGenerator = basin.Basin(machineInfo)
fimexGenerator.instantiate(fabfile='fabfile_fimex.py')
print("The ip of the google cloud instance is {}".format(fimexGenerator.ip))

#Need to wait until the instance is up and running. There must be a better way.
time.sleep(10)

#Testing connection to instance
fimexGenerator.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**fimexGenerator.machineInfo))

## Creating cfg file for download

In [ ]:
cfgTemplate='''[extract]
reduceToBoundingBox.south={south}
reduceToBoundingBox.north={north}
reduceToBoundingBox.west={west}
reduceToBoundingBox.east={east}
selectVariables={variable}
'''

## Getting all variables for all basins using fimex

In [ ]:
#Getting all boundaries to download:
with open(os.path.join(geoResultsFolder,'box.txt'), 'r') as f:
    lines = f.read().splitlines()
boxes = [parse('BOX({west} {south},{east} {north})',i.split(';')[1]) for i in lines]
names =  [re.split(';|,|\s',i)[0] for i in lines]
# import os
for i in varsToGet:
    #Getting only dates from 2010 to 2016
    args = [os.path.join(listFolder, 
                         i + '_' + model
                        ), 
            "2014", "2015"
           ]
    !!cat {args[0]}.txt | grep -E `seq -s \| {args[1]} {args[2]}` > {args[0]}_mini.txt
#     'head -n 40 {0}.txt > {0}_mini.txt'.format(i + '_' + model)
    currentDict = boxes[0].named
    currentDict['variable'] = i
    with open('box.cfg','w') as f:
        f.write(cfgTemplate.format(**currentDict))
    for j in names:
        fimexGenerator.getNcDataForBox(os.path.join(listFolder, i + '_' + model) + '_mini.txt',
                                  'box.cfg',
                                  i + '_' + j + '.nc'
                                  )   

In [ ]:
fimexGenerator.kill()

In [ ]:
a=b=2
print(a,b)

In [ ]:
print(fimexGenerator.initString)